# Demos: Lecture 11

## Demo 1: QFT from scratch

<img src="fig/qft-full-screencap.png" width="500px">

$$
R_k = \begin{pmatrix} 1 & 0 \\ 0 & e^{2\pi i/2^k} \end{pmatrix}
$$

In [6]:
import pennylane as qml
import numpy as np

/opt/conda/envs/pennylane/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [7]:
def apply_swaps(wires=None):
    num_wires = len(wires)
    for wire in range(num_wires // 2):
        qml.SWAP(wires=[wire, num_wires - wire - 1])

In [8]:
def apply_controlled_rotations(target_idx=None, wires=None):
    qml.Hadamard(wires=target_idx)
    
    num_wires = len(wires)
    num_shifts = num_wires - target_idx + 1
    
    phase_shifts = [2 * np.pi / 2 ** k for k in range(2, num_shifts)]
    
    for idx, shift in enumerate(phase_shifts):
        control_idx = target_idx + idx + 1
        qml.ControlledPhaseShift(shift, wires=[control_idx, target_idx])

In [9]:
def QFT_circuit():
    for target_idx in range(len(dev.wires)):  
        apply_controlled_rotations(target_idx=target_idx, wires=dev.wires)
    
    apply_swaps(wires=dev.wires)
    
    return qml.state()

In [10]:
dev = qml.device('default.qubit', wires=3)

In [11]:
qnode = qml.QNode(QFT_circuit, dev)

In [12]:
print(qml.draw(qnode)())

 0: ──H──╭ControlledPhaseShift(1.57)─────╭ControlledPhaseShift(0.785)──────────────────────────────────╭SWAP──╭┤ State 
 1: ─────╰ControlledPhaseShift(1.57)──H──│─────────────────────────────╭ControlledPhaseShift(1.57)─────│──────├┤ State 
 2: ─────────────────────────────────────╰ControlledPhaseShift(0.785)──╰ControlledPhaseShift(1.57)──H──╰SWAP──╰┤ State 



In [13]:
print(qnode())

[0.35355339+0.j 0.35355339+0.j 0.35355339+0.j 0.35355339+0.j
 0.35355339+0.j 0.35355339+0.j 0.35355339+0.j 0.35355339+0.j]


## Demo 2: QFT from template

In [14]:
@qml.qnode(dev)
def QFT_with_template():
    qml.QFT.decomposition(wires=dev.wires)
    return qml.state()

In [15]:
print(qml.draw(QFT_with_template)())

 0: ──H──╭ControlledPhaseShift(1.57)─────╭ControlledPhaseShift(0.785)──────────────────────────────────╭SWAP──╭┤ State 
 1: ─────╰ControlledPhaseShift(1.57)──H──│─────────────────────────────╭ControlledPhaseShift(1.57)─────│──────├┤ State 
 2: ─────────────────────────────────────╰ControlledPhaseShift(0.785)──╰ControlledPhaseShift(1.57)──H──╰SWAP──╰┤ State 



## Demo 3: Quantum phase estimation

<img src="fig/qpe-full-screencap.png" width="500px">

In [16]:
U = np.kron(qml.T._matrix(), np.kron(qml.PauliX._matrix(), np.eye(2)))

In [17]:
eigvals, eigvecs = np.linalg.eig(U)

In [18]:
eigvals

array([ 1.        +0.j        , -1.        +0.j        ,
        1.        +0.j        , -1.        +0.j        ,
        0.70710678+0.70710678j, -0.70710678-0.70710678j,
        0.70710678+0.70710678j, -0.70710678-0.70710678j])

In [19]:
target_eigvec = eigvecs[:, 4]

In [20]:
target_eigvec

array([-0.        +0.00000000e+00j, -0.        +0.00000000e+00j,
       -0.        +0.00000000e+00j, -0.        +0.00000000e+00j,
        0.70710678+0.00000000e+00j, -0.        +0.00000000e+00j,
        0.70710678+5.55111512e-17j, -0.        +0.00000000e+00j])

In [21]:
np.dot(U, target_eigvec)

array([0. +0.j , 0. +0.j , 0. +0.j , 0. +0.j , 0.5+0.5j, 0. +0.j ,
       0.5+0.5j, 0. +0.j ])

In [22]:
eigvals[4] * target_eigvec

array([-0. +0.j , -0. +0.j , -0. +0.j , -0. +0.j ,  0.5+0.5j, -0. +0.j ,
        0.5+0.5j, -0. +0.j ])

In [23]:
eigvals[4]

(0.7071067811865475+0.7071067811865475j)

In [24]:
np.angle(eigvals[4]) / (2 * np.pi)

0.125

In [25]:
estimation_wires = range(3)
target_wires = range(3, 6)

dev = qml.device('default.qubit', wires=len(estimation_wires)+len(target_wires), shots=1)

@qml.qnode(dev)
def quantum_phase_estimation():
    # Prepare uniform superposition
    for wire in estimation_wires:
        qml.Hadamard(wires=wire)
        
    # Prepare eigenstate on target register
    qml.MottonenStatePreparation(target_eigvec, wires=target_wires)
    
    # Apply controlled unitaries
    for est_wire in estimation_wires:
        power = 2 ** (len(estimation_wires) - est_wire - 1)
        qml.ControlledQubitUnitary(
            np.linalg.matrix_power(U, power),
            control_wires=est_wire,
            wires=target_wires)
            
    qml.adjoint(qml.QFT)(wires=estimation_wires)
    
    return qml.sample(wires=estimation_wires)

In [26]:
quantum_phase_estimation()

tensor([0, 0, 1], requires_grad=True)

In [27]:
print(qml.draw(quantum_phase_estimation)())

 0: ───H─────────────────────────────╭C─────────────╭QFT⁻¹──╭┤ Sample[basis] 
 1: ───H─────────────────────────────│────╭C────────├QFT⁻¹──├┤ Sample[basis] 
 2: ───H─────────────────────────────│────│────╭C───╰QFT⁻¹──╰┤ Sample[basis] 
 3: ──╭MottonenStatePreparation(M0)──├U0──├U1──├U2───────────┤               
 4: ──├MottonenStatePreparation(M0)──├U0──├U1──├U2───────────┤               
 5: ──╰MottonenStatePreparation(M0)──╰U0──╰U1──╰U2───────────┤               
U0 =
[[ 1.+0.0000000e+00j  0.+0.0000000e+00j  0.+0.0000000e+00j
   0.+0.0000000e+00j  0.+0.0000000e+00j  0.+0.0000000e+00j
   0.+0.0000000e+00j  0.+0.0000000e+00j]
 [ 0.+0.0000000e+00j  1.+0.0000000e+00j  0.+0.0000000e+00j
   0.+0.0000000e+00j  0.+0.0000000e+00j  0.+0.0000000e+00j
   0.+0.0000000e+00j  0.+0.0000000e+00j]
 [ 0.+0.0000000e+00j  0.+0.0000000e+00j  1.+0.0000000e+00j
   0.+0.0000000e+00j  0.+0.0000000e+00j  0.+0.0000000e+00j
   0.+0.0000000e+00j  0.+0.0000000e+00j]
 [ 0.+0.0000000e+00j  0.+0.0000000e+00j  0.+0.0000